In [1]:
import numpy as np
import pandas as pd
import pickle
from datetime import timedelta, datetime
from dateutil.relativedelta import relativedelta
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

In [2]:
model = LinearDiscriminantAnalysis
coin_names = ['Bitcoin', 'Litecoin','Ethereum', 'Dogecoin', 'XRP']

Part 1 - Define different lag days

In [3]:
def lagged_data(coin_names,lag_day):
    X = pd.DataFrame()
    y = pd.DataFrame()
    for coin_name in coin_names:
        twitter_signals = pd.read_csv('{0}_twitter_signals.csv'.format(coin_name))
        twitter_signals['Date'] = twitter_signals['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
        twitter_signals['Date'] = twitter_signals['Date'].apply(lambda x: x.strftime('%m-%d-%Y'))
        news_signals = pd.read_csv('{0}_news_signals.csv'.format(coin_name))
        # news_signals.drop(columns = ['compound','positive','neutral','negative','polarity','subjective'], inplace=True)
        # twitter_signals.drop(columns = ['label','Returns','Adj Close'], inplace=True)
        combined_signals = news_signals.merge(twitter_signals, left_on='Date', right_on='Date')
        
        X1 = combined_signals.drop(columns = ['Date', 'label','Returns','Adj Close'])
        X1 = X1.shift(lag_day)
        y1 = combined_signals[['label']]
        X = X.append(X1.iloc[lag_day:,:], ignore_index=True)
        y = y.append(y1.iloc[lag_day:,:], ignore_index=True)
    return X,y

In [4]:
# there is already one day lag (so 0 means 1 day lag)
for lag_day in range(0,7):
    X, y = lagged_data(coin_names,lag_day)
    # Define method to evaluate model
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
    # Evaluate model
    scores = cross_val_score(model(), X.iloc[lag_day:,:], y[lag_day:], scoring='accuracy', cv=cv, n_jobs=-1)
    losses = cross_val_score(model(), X.iloc[lag_day:,:], y[lag_day:], scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
    cor_lag_day = lag_day+1
    print('lag day:' + str(cor_lag_day))
    print('Mean Accuracy for: %.3f (+- %.3f)' % (np.mean(losses), np.std(losses)))
    print('Mean Accuracy for: %.3f (+- %.3f)' % (np.mean(scores), np.std(scores)))
    print('--------------------------------------------------')

lag day:1
Mean Accuracy for: -0.371 (+- 0.076)
Mean Accuracy for: 0.645 (+- 0.067)
--------------------------------------------------
lag day:2
Mean Accuracy for: -0.403 (+- 0.076)
Mean Accuracy for: 0.635 (+- 0.067)
--------------------------------------------------
lag day:3
Mean Accuracy for: -0.403 (+- 0.100)
Mean Accuracy for: 0.641 (+- 0.055)
--------------------------------------------------
lag day:4
Mean Accuracy for: -0.402 (+- 0.086)
Mean Accuracy for: 0.639 (+- 0.051)
--------------------------------------------------
lag day:5
Mean Accuracy for: -0.455 (+- 0.108)
Mean Accuracy for: 0.616 (+- 0.060)
--------------------------------------------------
lag day:6
Mean Accuracy for: -0.475 (+- 0.117)
Mean Accuracy for: 0.590 (+- 0.060)
--------------------------------------------------
lag day:7
Mean Accuracy for: -0.394 (+- 0.066)
Mean Accuracy for: 0.616 (+- 0.050)
--------------------------------------------------
